<a href="https://colab.research.google.com/github/AhmadMujtaba200210/5-Open-Ai-Projects/blob/develop/intel_text_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Dec  3 11:02:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://www.github.com/huggingface/transformers

!pip install git+https://github.com/huggingface/accelerate

  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-0t0k7dk7
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-0t0k7dk7
  Resolved https://www.github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-jwg2jq_u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-jwg2jq_u
  Resolved https://github.com/huggingface/accelerate to commit 04825483637a002deed91602878efbc1e4cfc0b4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.25.0.dev0-py3-none-any.whl size=26

In [ ]:
!pip install -q peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

In [ ]:
import os
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline

#Free Google Colab offers a 15GB Graphics Card (Limited Resources --> Barely enough to store Llama 2–7b’s weights)

#We also need to consider the overhead due to optimizer states, gradients, and forward activations


#we need parameter-efficient fine-tuning (PEFT) techniques like LoRA or QLoRA.

#To drastically reduce the VRAM usage, we must fine-tune the model in 4-bit precision, which is why we’ll use QLoRA here.



In [ ]:
import transformers

In [ ]:
#model_name = "NousResearch/Llama-2-7b-chat-hf"

model_name = 'Intel/neural-chat-7b-v3-1'

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

fp16 = False
bf16 = False

device_map = {"": 0} ## LOAD THE ENTIRE MODLE ON THE GPU

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, max_length=1000, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]


In [ ]:
system_input = "You are a food recipe generator. Your mission is to give food recipe in following steps, step1: title, step2: ingredients, step3: instructions to prepare this recipe."
user_input = "1 lb ground beef, 1 onion (minced), 2 cloves garlic (minced), 1 carrot (grated), 1 celery stalk (chopped), 1 can (14 oz) crushed tomatoes, 2 tbsp tomato paste, 1 cup beef broth, 1/2 cup red wine (optional), 1 tsp dried oregano, 1 tsp dried basil, Salt and pepper to taste, 2 tbsp olive oil, 1 lb spaghetti, Parmesan cheese."
response = generate_response(system_input, user_input)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Title: Beefy Spaghetti with Tomato-Herb Sauce

Step2: Ingredients
- 1 lb ground beef
- 1 onion (minced)
- 2 cloves garlic (minced)
- 1 carrot (grated)
- 1 celery stalk (chopped)
- 1 can (14 oz) crushed tomatoes
- 2 tbsp tomato paste
- 1 cup beef broth
- 1/2 cup red wine (optional)
- 1 tsp dried oregano
- 1 tsp dried basil
- Salt and pepper to taste
- 2 tbsp olive oil
- 1 lb spaghetti
- Parmesan cheese

Step3: Instructions
1. In a large skillet, heat 2 tbsp olive oil over medium heat. Add the ground beef and cook until browned, breaking it up into small pieces.
2. Add the minced onion, garlic, carrot, and celery to the skillet. Cook for about 5 minutes, stirring occasionally, until the vegetables are softened.
3. Pour in the crushed tomatoes, tomato paste, beef broth, red wine (if using), dried oregano, and dried basil. Stir well to combine.
4. Season with salt and pepper to taste. Bring the mixture to a simmer and cook for about 20 minutes, stirring occasionally.
5. While the sauce is